In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
#from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import sys
import time
import pickle
import glob
import matplotlib.pyplot as plt
import tqdm

In [ ]:
## Initialize class for predictor functions
class ReturnPredictions(object):
    def __init__(self,cfg):
        self.predictor = DefaultPredictor(cfg)
        self.metadata = MetadataCatalog.get(
                cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused")

    def _frame_from_video(self,video):
        while video.isOpened():
            success,frame = video.read()
            if success:
                yield frame
            else:
                break

    def run_on_video(self,video):
        frame_gen = self._frame_from_video(video)
        for frame in frame_gen:
            yield self.predictor(frame)

    def run_on_image(self,image):
      # Convert image from OpenCV BGR format to Matplotlib RGB format.
      yield self.predictor(image)


## Load and setup model and predictor
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

predictor = ReturnPredictions(cfg)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
rootPath = '/content/drive/My Drive/DL_CV_FinalProject/'

## Check for GPU 
gpu_check = torch.cuda.is_available()
print(gpu_check)

In [ ]:

videoID = 'fall-01-cam0-rgb'
allImgNames=sorted(glob.glob('/content/drive/My Drive/DL_CV_FinalProject/'+ videoID + '/*.png'))

testImg = cv2.imread(allImgNames[0])
print(testImg.shape)

plt.imshow(testImg)

In [ ]:
storeKeypoints = np.zeros((len(allImgNames),17,3))

imgID = 1
for i in range(len(allImgNames)):
  #print('\n----------')
  print('----Image #:', imgID)
  fname = allImgNames[i]
  img = cv2.imread(fname)
  # if running on video change the line below to "run_on_video"
  output_generator = predictor.run_on_image(img)

  for output in output_generator:
    countOuts += 1
    imgH,imgW = output['instances'].image_size
    personScore = output['instances'].scores.cpu().numpy()
    keypoints_all = output['instances'].pred_keypoints.cpu().numpy()
    person_scores = output['instances'].scores.cpu().numpy()
    idx_scores_valid = np.argwhere(person_scores>0.75)

    imgID += 1


  if len(idx_scores_valid) == 0:
    print('Could not detect person!')
    print(fname)
  elif len(idx_scores_valid) == 1:
    #print('Valid case')
    storeKeypoints[i] = keypoints_all[idx_scores_valid]
  elif len(idx_scores_valid) > 1:
    print('ERROR: More than one person detected! This script is only valid for videos with 1 person so selecting person with max score. ')
    print(fname)
    storeKeypoints[i] = keypoints_all[np.argmax(person_scores)]
  
  try:
    print("All Scores:",person_scores)
    imgCopy = img.copy()
    for key in keypoints_all:
      for pt in key:
        cv2.circle(imgCopy, (pt[0],pt[1]), 3, (255,0,0), -1)

    plt.imshow(imgCopy)
    plt.show()
  except: 
    print("All Scores:",person_scores)
    print('Could not draw points')
  
#### From detectron2 Code ####
# COCO_PERSON_KEYPOINT_NAMES = (
#     "nose",
#     "left_eye", "right_eye",
#     "left_ear", "right_ear",
#     "left_shoulder", "right_shoulder",
#     "left_elbow", "right_elbow",
#     "left_wrist", "right_wrist",
#     "left_hip", "right_hip",
#     "left_knee", "right_knee",
#     "left_ankle", "right_ankle",
# )

In [ ]:
savePath = rootPath + 'keypoints/'
np.save(savePath + videoID + '.npy',storeKeypoints)